In [1]:
import pandas as pd

Скрипт предназначен для извлечения причины запрета лекарства из приказа о ее запрете.

Мы хотим построить алгоритм отделяющий "общие слова" от настоящей причины о запрете. 

Для некоторых лекарственных средств, мы знаем причины из ручной разметки, и в основном она  описана одним предложением.
Для обучения алгоритма машинного обучения кроме положительных примеров хотелось бы иметь и отрицательные.
В то же время текст приказа довольно объемный, поэтому случайно взятое предложение из приказа почти наверное будет "не причиной" о запрете, это дает нам право восполнить выборку отрицательными примерами.

После обучения алгоритма, на этапе предсказания он применяется к каждому из предложений приказа, и выбирается предложение с наибольшей вероятностью нахождения в нем причины о запрете.

# считаем данные

In [2]:
a = pd.read_csv('./med_df_text.csv')

In [3]:
b = pd.read_excel('./Report_Database.xlsx')

# предобработка

тут мы разбиваем тексты из приказа на предложения

In [4]:
from nltk.tokenize import sent_tokenize

# разиваем текст на предложения
a['preprocessed_order'] = a['Приказ'].fillna('').apply(sent_tokenize)

In [5]:
# функция которая сделает каждое предложение из текста объектом
def unlistify(df, column):
    matches = [i for i,n in enumerate(df.columns)
             if n==column]

    if len(matches)==0:
        raise Exception('Failed to find column named ' + column +'!')
    if len(matches)>1:
        raise Exception('More than one column named ' + column +'!')

    col_idx = matches[0]

      # Helper function to expand and repeat the column col_idx
    def fnc(d): 
        row = list(d.values[0])
        bef = row[:col_idx]
        aft = row[col_idx+1:]
        col = row[col_idx]
        z = [bef + [c] + aft for c in col]
        return pd.DataFrame(z)

    col_idx += len(df.index.shape) # Since we will push reset the index
    index_names = list(df.index.names)
    column_names = list(index_names) + list(df.columns)
    return (df
          .reset_index()
          .groupby(level=0,as_index=0)
          .apply(fnc)
          .rename(columns = lambda i :column_names[i])
          .set_index(index_names)
          )



In [6]:
a = unlistify(a, 'preprocessed_order').reset_index()

In [7]:
test_set = a[['index','preprocessed_order']].copy()

In [8]:
train_set = b.reset_index()[['index','Причины забраковки']].rename(columns={'Причины забраковки':'preprocessed_order'})

In [9]:
test_set['target'] = 0
train_set['target'] = 1

In [10]:
full = pd.concat([train_set, test_set]).fillna('')

In [11]:
import pymystem3

In [12]:
stemmer = pymystem3.Mystem()

In [13]:
full['stemmed_order'] = full['preprocessed_order'].apply(lambda x: ''.join(stemmer.lemmatize(x)))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
vect = TfidfVectorizer(ngram_range=(1,3))

In [16]:
full_tfidf = vect.fit_transform(full['stemmed_order'])

/Users/kolya/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [17]:
import lightgbm as lgb
# lgb.LG.fit(full)

In [18]:
clf = lgb.LGBMClassifier().fit(full_tfidf, full['target'])

In [19]:
p = clf.predict_proba(full_tfidf[train_set.shape[0]:])[:, 1]

In [21]:
test_set.shape

(68533, 3)

In [ ]:
test_set['pred'] = p

# проверим как это сработало, на случайных лекарствах

In [43]:
indexes = get_random() # выберем случайные приказы

In [44]:
quality_check = test_set[test_set['index'].isin(indexes)]

In [52]:
quality_check['max_pred'] = quality_check.groupby('index').pred.transform('max')

/Users/kolya/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
answers = quality_check[quality_check['pred'] == quality_check['max_pred']]

In [55]:
for i in answers.iterrows():
    print(i[1].preprocessed_order)

Натрия дисульфит», «Количественное определение.
Видимые частицы», «Количественное определение.
Климовой» (Россия) в  связи  с  развитием  нежелательной  реакции.
Видимые частицы» - серии 290515.2.
Умет, Тамбовская  область), показатель  «Упаковка» (отсутствует  вторичная  упаковка) — серии  031115.
